# Introduction to Prompting

"Prompting" is the way in which humans communicate with an AI using human language to instruct the LLM systems. 

Prompt engineering is a discipline that creates and optimizes prompts to leverage language models. A prompt engineer translates an idea from conversational language to more precised and optimized instructions to an AI.

In this Jupyter Notebook we provide a complete guide to prompting.

# Table of Contents
- [Introduction](#introduction)
- [Creating your first prompt](#first-prompt)
- [Prompting Techniques](#prompting-techniques)
    - [Zero Shot Prompting](#zero-shot-prompts)
    - [In-Context Learning and Few-Shot Prompting](#context-few-shot)
        - [In-Context Learning](#context-prompting)
        - [Few-Shot Prompting](#few-shot-prompting)

# Introduction
<a id="introduction"></a>

For a new project we import OpenAI from openai. This will automatically defaults to getting the key using **<u>os.environ.get("OPENAI_API_KEY")</u>**, it's important that for this to automatically work your key must be named OPENAI_API_KEY. 

**Note:** If the code doesn't work, view Readme.md on how to set up this project.

In [7]:
from openai import OpenAI

client = OpenAI()

# Creating your first prompt
<a id="first-prompt"></a>

In this example we have a few lines of code:
- prompt_system: we explain the system what role we expect it to take
- prompt: we give instructions on what we want to do
- response: we connect to OpenAIs API and give the model instructions

### Example: Story teller

In [8]:
prompt_system = "You are a helpful assistant whose goal is to help write stories."
prompt = """Continue the following story. Write no more than 50 words.

Once upon a time, in a world where animals could speak, a courageous mouse named
Benjamin decided to"""

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": prompt_system},
        {"role": "user", "content": prompt}
        ])

print(response.choices[0].message.content)

embark on a dangerous journey to find the legendary Cheese Mountain. With his tiny walking stick in hand, he faced countless challenges, including cunning cats and treacherous traps. But Benjamin pressed on, determined to uncover the secret of the elusive mountain.


# Prompting Techniques
<a id="prompting-techniques"></a>

### Zero Shot Prompting
<a id="zero-shot-prompts"></a>

**Zero Shot Prompting** is when a model is asked to produce output with our examples demonstrating the task. We tested it in the previous example. Here is another example.

This example has a variable ***{topic}***. This would be an input from a program that someone can add of a poem writer assistant. We will add the variable at the beggining of the code and you can edit it from there. In future notebooks we will learn how to create our LLM.

In [9]:
topic = "soul"
# This is the prompt, we use the topic variable that will be our input somewhere where we implement our model
#
prompt_system = "You are a helpful assistant whose goal is to write short poems."
prompt = f"""Write a short poem about {topic}."""

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": prompt_system},
        {"role": "user", "content": prompt}
        ])

print(response.choices[0].message.content)

In the depths of the heart, a spark ignites,  
A precious essence, a guiding light,  
The soul that dances, pure and true,  
Connecting us to all that's new.  
So cherish your soul, let it shine bright,  
For it's a beacon in the darkest night.


# In-Context Learning and Few-Shot Prompting
<a id="context-few-shot"></a>

**In context learning** is an approach where a model learns from examples or demonstrations in the prompt.

**Few-Shot prompting** is a subset of in context learning that presents the model with a small set of relevant examples. This is useful for more complex tasks.

Unlike zero shot, we use examples to improve performance.

## In-Context learning
<a id="context-prompting"></a>

As you can see in this example we provide three examples with three verses, if you see the output it follows the same structure, guiding itself from examples.

In [11]:
input_topic = "love"

prompt_system = "You are a helpful assistant whose goal is to write short poems."
prompt = f"""Write a short poem about {topic}."""
examples = {"nature": """Birdsong fills the air,\nMountains high and valleys deep,\nNature's music sweet.""", 
            "winter": """Snow blankets the ground,\nSilence is the only sound,\nWinter's beauty found."""
            }

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": prompt_system},
        {"role": "user", "content": prompt.format(topic="nature")},
        {"role": "assistant", "content": examples["nature"]},
        {"role": "user", "content": prompt.format(topic="winter")},
        {"role": "assistant", "content": examples["winter"]},
        {"role": "user", "content": prompt.format(topic=input_topic)}
        ])

print(f"Output:\n\n{response.choices[0].message.content}\n")

Output:

In the heart's quiet space,
The soul finds its resting place,
Guiding with gentle grace.



## Few-Shot prompting
<a id="few-shot-prompting"></a>

For this example that is more complex we will use LangChain, this is a library designed to facilitate use of LLMs. We will go further into the topic in a future notebook.

You can see the printed prompt to get an idea of how it works by putting together all this code into a final prompt.

**Note:** While few-shot learning is effective, it encounters challenges, mainly when tasks are complex. More advanced strategies, like chain-of-thought prompting, become increasingly valuable in such cases. This technique breaks down complex problems into simpler phases, offering examples for each stage and enhancing the model’s logical reasoning capacity.

In [13]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI

# Initialize LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

examples = [
    {"color": "red", "emotion": "passion"},
    {"color": "blue", "emotion": "serenity"},
    {"color": "green", "emotion": "tranquility"},
]

example_formatter_template = """
Color: {color}
Emotion: {emotion}
"""

example_prompt = PromptTemplate(
    input_variables=["color", "emotion"],
    template=example_formatter_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="""Here are some examples of colors and the emotions associated with \
them:""",
    suffix="""Now, given a new color, identify the emotion associated with \
it:\nColor: {input}\nEmotion:""",
    input_variables=["input"],
    example_separator="\n",
)

formatted_prompt = few_shot_prompt.format(input="purple")

# Create the LLMChain for the prompt
chain = LLMChain(llm=llm, prompt=PromptTemplate(template=formatted_prompt,
input_variables=[]))

# Run the LLMChain to get the AI-generated emotion associated with the input
# color
response = chain.run({})

# We print the example of the prompt sent
print("Input: 'Color: purple'")
print("Output: 'Emotion:", f"{response}'")
print(f"\nPrompt:\n\n{chain.prompt.template}")

Input: 'Color: purple'
Output: 'Emotion: royalty'

Prompt:

Here are some examples of colors and the emotions associated with them:

Color: red
Emotion: passion


Color: blue
Emotion: serenity


Color: green
Emotion: tranquility

Now, given a new color, identify the emotion associated with it:
Color: purple
Emotion:


## Role Prompting

Role prompting involves instructing the LLM to assume a specific role or identity for task execution, such as functioning as a copywriter. This instruction can influence the model’s response by providing context or perspective for the task. When working with role prompts, the iterative process includes:

1. Defining the role in the prompt. For example, “As a copywriter, create engaging catchphrases for AWS services.”
2. Utilizing the prompt to generate a response from an LLM.
3. Evaluating the response and refining the prompt as needed for improved outcomes".

In [17]:
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI

# Initialize LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

template = """
As a futuristic robot band conductor, I need you to help me come up with a song title.
What's a cool song title for a song about {theme} in the year {year}?"""

prompt = PromptTemplate(
    input_variables=["theme", "year"],
    template=template,
    )

# Input data for the prompt
input_data = {"theme": "interstellar travel", "year": "3030"}

# Create LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the LLMChain to get the AI-generated song title
response = chain.run(input_data)

print("Theme: interstellar travel")
print("Year: 3030")
print("AI-generated song title:", response)

Theme: interstellar travel
Year: 3030
AI-generated song title: "Galactic Odyssey: Journey Through the Stars in 3030"


## What makes a good prompt?

**Precise Directions:** The prompt is structured as a straightforward request for generating a song title, explicitly stating the context: “As a futuristic robot band conductor.” This clarity helps the LLM recognize that the output should be a song title linked to a futuristic context.

**Specificity:** The prompt calls for a song title connected to a particular theme and year, “{theme} in the year {year}.” This level of detail allows the LLM to produce a relevant and imaginative response. The flexibility of the prompt to accommodate various themes and years through input variables adds to its versatility and applicability.

**Promoting Creativity:** The prompt does not restrict the LLM to a specific format or style for the song title, encouraging a wide range of creative responses based on the specified theme and year.

**Concentrated on the Task:** The prompt concentrates exclusively on creating a song title, simplifying the LLM process to deliver an appropriate response without being diverted by unrelated subjects. Integrating multiple tasks in one prompt can confuse the model, potentially compromising its effectiveness in each task.